# Creating Relations in SQLite using Oscar nominations dataset

In [44]:
import sqlite3

In [45]:
conn = sqlite3.connect("nominations.db")

In [46]:
def query(s):
    result = conn.execute(s).fetchall()
    for item in result:
        print(item)
    return

In [47]:
query("PRAGMA table_info(nominations);")

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [48]:
query("SELECT * FROM nominations LIMIT 10;")

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


In [60]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

conn.execute("CREATE TABLE ceremonies(id integer PRIMARY KEY, Year ineger, Host text);")
conn.executemany("INSERT INTO ceremonies (Year, Host) VALUES (?, ?);", years_hosts)
query("PRAGMA table_info(ceremonies);")


(0, 'id', 'integer', 0, None, 1)
(1, 'Year', 'ineger', 0, None, 0)
(2, 'Host', 'text', 0, None, 0)


In [61]:
query("SELECT * FROM ceremonies LIMIT 10;")

(1, 2010, 'Steve Martin')
(2, 2009, 'Hugh Jackman')
(3, 2008, 'Jon Stewart')
(4, 2007, 'Ellen DeGeneres')
(5, 2006, 'Jon Stewart')
(6, 2005, 'Chris Rock')
(7, 2004, 'Billy Crystal')
(8, 2003, 'Steve Martin')
(9, 2002, 'Whoopi Goldberg')
(10, 2001, 'Steve Martin')


In [62]:
conn.execute('PRAGMA foreign_keys = ON;')

In [64]:
conn.execute('CREATE TABLE nominations_two(id integer PRIMARY KEY, category text, nominee text, movie text,\
character text, won integer, ceremony_id integer, FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id));')

In [65]:
joined_nominations = conn.execute("SELECT nominations.category, nominations.nominee, nominations.movie,\
nominations.character, nominations.won, ceremonies.id FROM nominations INNER JOIN ceremonies ON \
nominations.year == ceremonies.year;")

In [69]:
conn.executemany("INSERT INTO nominations_two (category, nominee, movie, character, won, \
ceremony_id) VALUES (?,?,?,?,?,?);", joined_nominations)

In [72]:
query("SELECT * FROM nominations_two LIMIT 5;")

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)


In [74]:
conn.execute("DROP TABLE nominations;")

In [75]:
conn.execute("ALTER TABLE nominations_two RENAME TO nominations;")

In [80]:
conn.execute("CREATE TABLE movies(id integer PRIMARY KEY, movie text);")

In [81]:
conn.execute("CREATE TABLE actors(id integer PRIMARY KEY, actor text);")

In [91]:
conn.execute("CREATE TABLE movies_actors(id integer PRIMARY KEY, movie_id integer REFERENCES movies(id), \
actor_id integer REFERENCES actors(id));")

In [86]:
conn.execute("INSERT INTO movies (movie) SELECT DISTINCT movie FROM nominations;")
query("PRAGMA table_info(movies);")
query("SELECT * FROM movies LIMIT 5;")

(0, 'id', 'integer', 0, None, 1)
(1, 'movie', 'text', 0, None, 0)
(1, 'Biutiful')
(2, 'True Grit')
(3, 'The Social Network')
(4, "The King's Speech")
(5, '127 Hours')


In [87]:
conn.execute("INSERT INTO actors (actor) SELECT DISTINCT nominee FROM nominations;")
query("PRAGMA table_info(actors);")
query("SELECT * FROM actors LIMIT 5;")

(0, 'id', 'integer', 0, None, 1)
(1, 'actor', 'text', 0, None, 0)
(1, 'Javier Bardem')
(2, 'Jeff Bridges')
(3, 'Jesse Eisenberg')
(4, 'Colin Firth')
(5, 'James Franco')


In [92]:
movie_actor_pairs = conn.execute("SELECT movie, nominee FROM nominations;").fetchall()

conn.executemany("INSERT INTO movies_actors (movie_id, actor_id) VALUES ((SELECT id FROM movies WHERE movie == ?), \
(SELECT id FROM actors WHERE actor == ?));", movie_actor_pairs)

query("PRAGMA table_info(movies_actors);")
query("SELECT * FROM movies_actors LIMIT 5;")

(0, 'id', 'integer', 0, None, 1)
(1, 'movie_id', 'integer', 0, None, 0)
(2, 'actor_id', 'integer', 0, None, 0)
(1, 1, 1)
(2, 2, 2)
(3, 3, 3)
(4, 4, 4)
(5, 5, 5)


In [96]:
query("SELECT actor, movie FROM movies, actors LIMIT 5;")

('Javier Bardem', 'Biutiful')
('Jeff Bridges', 'Biutiful')
('Jesse Eisenberg', 'Biutiful')
('Colin Firth', 'Biutiful')
('James Franco', 'Biutiful')


## Potential next steps

* What other datasets can we add to the database?
* Based on what you know, brainstorm how you would populate the join table and the linked tables using data from nominations.